In [3]:
# Implement RAG using Langchain (LC)
from langchain_openai import AzureOpenAIEmbeddings
from langchain.chat_models import AzureChatOpenAI
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery
from langchain.chains import LLMChain
from dotenv import load_dotenv
import os

load_dotenv(override=True)
endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
credential = AzureKeyCredential(os.environ["AZURE_SEARCH_ADMIN_KEY"]) if len(os.environ["AZURE_SEARCH_ADMIN_KEY"]) > 0 else DefaultAzureCredential()
index_name = os.environ["AZURE_SEARCH_INDEX"]
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_key = os.environ["AZURE_OPENAI_API_KEY"] if len(os.environ["AZURE_OPENAI_API_KEY"]) > 0 else None
azure_openai_embedding_deployment = os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"]
embedding_model_name = os.environ["AZURE_OPENAI_EMBEDDING_MODEL"]
azure_openai_api_version = os.environ["OPENAI_API_VERSION"]
azure_openai_chatgpt_deployment = os.environ["AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME"]
openai_api_type = os.environ["OPENAI_API_TYPE"]
# Setup Langchain
# Create an Embeddings Instance of Azure OpenAI
embeddings = AzureOpenAIEmbeddings(
    model=embedding_model_name,
    deployment=embedding_model_name,
    openai_api_type = openai_api_type,
    openai_api_version = azure_openai_api_version,
    azure_endpoint = azure_openai_endpoint,
    openai_api_key = azure_openai_key,
    embedding_ctx_length=8191,
    chunk_size=1000,
    max_retries=6
)

# Create a Completion Instance of Azure OpenAI
llm = AzureChatOpenAI(
    model="gpt-3.5-turbo",
    deployment_name = azure_openai_chatgpt_deployment,
    openai_api_type = openai_api_type,
    openai_api_version = azure_openai_api_version,
    azure_endpoint  = azure_openai_endpoint,
    openai_api_key = azure_openai_key,
    temperature=0.7,
    max_retries=6,
    max_tokens=4000
)

# Ask the question
question = "What software tools do we have?"

# Create a prompt template with variables, note the curly braces
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
    input_variables=["original_question","search_results"],
    template="""
    Question: {original_question}

    Do not use any other data.
    Only use the data below when responding.
    {search_results}
    """,
)

# Get Embedding for the original question
question_embedded=embeddings.embed_query(question)

# Search Vector Store
search_client = SearchClient(
    endpoint,
    index_name,
    credential
)
vector_query = VectorizedQuery(
    vector=question_embedded,
    k_nearest_neighbors=5,
    fields="contentVector"
)

# Execute the search
results = list(search_client.search(
    search_text="",
    include_total_count=True,
    vector_queries=[vector_query],
    select=["id", "content", "title"],
))

# Build the Prompt and Execute against the Azure OpenAI to get the completion
chain = LLMChain(llm=llm, prompt=prompt, verbose=True)
response = chain.run({"original_question": question, "search_results": results})
print(response)



> Entering new LLMChain chain...
Prompt after formatting:

    Question: What software tools do we have?

    Do not use any other data.
    Only use the movie data below when responding.
    [{'content': 'Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.', 'id': '9', 'title': 'Azure DevOps', '@search.score': 0.5787385, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}, {'content': 'Azure DevTest Labs is a fully